# Operação do modelo

Este notebook apresenta um exemplo de como um modelo salvo no MLFlow pode ser aplicado em um dado dataset do DOROTHY.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import lpsds.notebook
from scripts.operation import ProductionModelOperation

## Dados de entrada

Para a correta operação do modelo, se fazem necessárias as seguintes informações, que precisam ser fornecidas tais como especificadas abaixo.

* URL para o servior do MLFLOW (https://mlflow.lps.ufrj.br ou http://localhost:5000, por exemplo): esta URL deve estr definida numa variável de ambiente chamada "MLFLOW_TRACKING_URI"
* RUN ID do modelo: o run id do modelo de operação salvo no MLFlow. O modelo deve estar salvo numa pasta chamada "operation_model" no MLFlow.
* Dataset do DOROTHY: o nome do dataset a ser solicitado ao DOROTHY para que usas imagens sejam fornecidas ao classificador.

Neste exemplo, como estamos oeprando via notebbok, emularemos a passagem do run id do modelo e do dataset através das variáveis "validation_dataset_name"  e "mlflow_run_id".

Com respeito à saída do modelo, a mesma deve ser colocada na pasta /bucket/output da sua imagem Docker. A entrega do modelo deve ser sempre um pandas.DataFrame, salvo como parquet, contendo pelo menos 4 colunas:

* Nome do arquivo da imagem (identificador único)
* y_proba: o valor de saída modelo
* y_pred: a decisão do modelo (+1 p/ TB+ e 0 para TB-)

O nome do arquivo dese ser no formato {dataset_name}-output.parquet. Por exemplo, supondo a operação do modelo em imagens de um dataset chamado "china", o arquivo de saída deverá ser salvo em "/bucket/output/china-output.parque"

In [ ]:
device_type = 'gpu'
validation_dataset_name =  'china'
mlflow_run_id = 'bd6019f44da0472b8c444d9e83c7ac50'
operate = ProductionModelOperation(device_type=device_type)

## Carregando o dataset de validação

In [ ]:
X, metadata = operate.get_dataset_images(validation_dataset_name)

## Carregando o modelo armazenado no MLFlow

In [ ]:
model, decision_threshold = operate.get_operation_model(mlflow_run_id)

## Aplicando as imagens do dataset passado no modelo especificado

In [ ]:
y_proba, y_pred = operate.operate_model(model, X, decision_threshold)

In [ ]:
plt.figure()
fig, axs = plt.subplots(2,1)
sns.histplot(data=y_proba, ax=axs[0])
axs[0].set_title('Distribuição da Saída do Modelo');

sns.histplot(data=y_pred, ax=axs[1])
axs[1].set_title('Distribuição da Decisão do Modelo');

### Salvando o resultado

In [ ]:

operate.save_results('/bucket/temp/model', metadata, y_proba, y_pred)

## Calculando as estatísticas de execução

In [ ]:
time_df = operate.get_processing_stats(model, X)

### Salvando o resultado

In [ ]:
operate.save_results('/bucket/temp/model-stats', metadata, y_proba, y_pred)